In [23]:
#######################################################
##### STRIP TWEET #####################################
#######################################################

#import pandas as pd
#df = pd.read_csv("train2017.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
#corpus = df[3].tolist()

import pandas as pd
import re
import nltk
import string
#READ TRAINING SET
df = pd.read_csv("small_train.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
train_corpus = df[3].tolist()

#READ TEST SET
df = pd.read_csv("small_test.tsv", sep='\t', header=None)
#df[2].tolist() #sentiments
test_corpus = df[3].tolist()

translate_table = dict((ord(char), None) for char in string.punctuation)   

def clean_corpus(corpus):
    clean_corpus = []
    for tweet in corpus:
        tweet = re.sub(r"http\S+", "", tweet) #remove link
        tweet = ' '.join([word for word in tweet.split(' ')  if not word.startswith('@')])
        tweet = tweet.translate(translate_table) #remove symbols 
        tweet = tweet.lower()
        clean_corpus.append(tweet)
    return clean_corpus
    
clean_train_corpus = clean_corpus(train_corpus)
clean_test_corpus = clean_corpus(test_corpus)

print(clean_train_corpus)
print(clean_test_corpus)


['gas by my house hit 339 im going to chapel hill on sat  😂', 'theo walcott is still shit fan watch rafa and johnny deal with him on saturday', 'its not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general says israels iron dome cant deal with their missiles keep talking like that and we may end up finding out', 'tehran mon amour obama tried to establish ties with the mullahs  via no barack obama  vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mashed out to niggas in paris in the club while in paris as cliche as it may sound weouthere', 'larry bird is ranked 4th alltime not including lebron or kobe just sayin']
['arianagrande ari by ariana grande 80 full singer actress', 'ariana grande kiis fm yours truly cd listening party in burbank arianagrande', 'ariana grande white house easter egg roll in washington arianagrande', 'cd musics ariana grande sweet like candy 34 oz 100 ml sealed in box 100 authenic new

In [24]:
#######################################################
##### TOKENIZATION ####################################
#######################################################

from nltk import word_tokenize

def tokenize(clean_corpus):
    tokens = []
    for tweet in clean_corpus:
        token = []
        token = word_tokenize(tweet)
        tokens.append(token)
    return tokens

train_tokens = tokenize(clean_train_corpus)
test_tokens = tokenize(clean_test_corpus)

    
print(train_tokens)
print(test_tokens)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'going', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'is', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['its', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'says', 'israels', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missiles', 'keep', 'talking', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'finding', 'out'], ['tehran', 'mon', 'amour', 'obama', 'tried', 'to', 'establish', 'ties', 'with', 'the', 'mullahs', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mashed', 'out', 'to', 'niggas', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'as', 'cliche', 'as', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'is', 'ra

In [25]:
from nltk import pos_tag

from nltk.corpus import wordnet
from nltk.stem import  WordNetLemmatizer
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(tokens):
    tweets = []
    for token_list in tokens:
        lemmatized = []
        for word in token_list:
            lemmatized.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
        tweets.append(lemmatized)
    return tweets
    
train_tweets = lemmatize(train_tokens)
test_tweets = lemmatize(test_tokens)

print(train_tweets)
print(test_tweets)

[['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], ['theo', 'walcott', 'be', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], ['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], ['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], ['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], ['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], ['mash', 'out', 'to', 'nigga', 'in', 'paris', 'in', 'the', 'club', 'while', 'in', 'paris', 'a', 'cliche', 'a', 'it', 'may', 'sound', 'weouthere'], ['larry', 'bird', 'be', 'ranked', '4th', 'alltime

In [26]:
final_train_corpus = []
for tweet in train_tweets:
    final_train_corpus.append(" ".join(str(word) for word in tweet))

final_test_corpus = []
for tweet in test_tweets:
    final_test_corpus.append(" ".join(str(word) for word in tweet))

print(final_train_corpus)
print(final_test_corpus)

['gas by my house hit 339 im go to chapel hill on sat 😂', 'theo walcott be still shit fan watch rafa and johnny deal with him on saturday', 'it not that im a gsp fan i just hate nick diaz cant wait for february', 'iranian general say israel iron dome cant deal with their missile keep talk like that and we may end up find out', 'tehran mon amour obama try to establish tie with the mullah via no barack obama vote mitt romney', 'i sat through this whole movie just for harry and ron at christmas ohlawd 911', 'mash out to nigga in paris in the club while in paris a cliche a it may sound weouthere', 'larry bird be ranked 4th alltime not include lebron or kobe just sayin']
['arianagrande ari by ariana grande 80 full singer actress', 'ariana grande kiis fm yours truly cd listen party in burbank arianagrande', 'ariana grande white house easter egg roll in washington arianagrande', 'cd music ariana grande sweet like candy 34 oz 100 ml seal in box 100 authenic new', 'side to side sidetoside arian

In [5]:
#BAG-OF-WORDS VECTORIZATION
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_BOW_train = vectorizer.fit_transform(final_train_corpus)
X_BOW_test = vectorizer.fit_transform(final_test_corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray()[0])
#print(X_BOW.toarray())


In [6]:
#TF-IDF VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_TFIDF_train = vectorizer.fit_transform(final_train_corpus)
X_TFIDF_test = vectorizer.fit_transform(final_test_corpus)
#print(X_TFIDF.toarray())


In [7]:
#we have 3 options: word2vec, word2vec in sklearn, doc2vec
from gensim.test.utils import common_texts

#IMPORT WORD2VEC
from gensim.models import Word2Vec
nested_corpus = [final_train_corpus[i:i+1] for i in range(0, len(final_train_corpus), 1)]
model = Word2Vec(train_tokens, size=100, window=5, min_count=1, workers=4) #size of vector is 100
model.train(train_tokens, total_examples=model.corpus_count,epochs=model.epochs)  # train word vectors
print(model['gas'])

#IMPORT WORD2VEC LIBRARY AVAILABLE IN SKLEARN

#from gensim.sklearn_api import W2VTransformer
#total = sum(tokens,[])
#vectorizer = W2VTransformer(size=10, min_count=1, seed=1)
#print(corpus)
#X_embeddings = vectorizer.fit(total).transform(tokens)



[-0.00455867 -0.00273707  0.00476938  0.00052254 -0.00099771 -0.00162601
  0.00435989 -0.00124946 -0.00384014 -0.00458047 -0.00040402  0.0014445
 -0.00472712  0.00038409  0.00052129 -0.00129615 -0.00039701 -0.00396524
 -0.00100313  0.00212705  0.00377884  0.00226566  0.00217207  0.00166716
  0.00113616 -0.00450988 -0.00040978  0.00431388 -0.00384496  0.00311142
  0.0048007   0.00244925 -0.00294611 -0.00133667 -0.0024853  -0.00407324
 -0.00181031  0.00403763 -0.00150548  0.00147443 -0.0046243   0.00141982
 -0.00314493  0.00193791 -0.00365485 -0.00257953 -0.00024534 -0.00088908
 -0.00291855  0.00285258 -0.00436966 -0.00362688 -0.00092549  0.00479371
  0.00423948  0.00073345  0.00135342  0.00307727  0.00446421 -0.00186245
 -0.003984    0.00499631 -0.00047878  0.00187234 -0.00466422  0.00281243
 -0.00363314 -0.00319005 -0.00411249  0.00271005  0.00340117 -0.0015745
 -0.00422078 -0.00123327  0.00023326  0.00222177  0.0047376  -0.00430097
  0.00268178  0.00182858  0.00334468 -0.00127572 -0.0

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [29]:
#Doc2vec is equal with word2vec but is more appropriate for phrases (vectorize phrases instead of words). I.e 
#1.Manos leaves the office every day at 18:00 to catch his train
#2. This season is called Fall, because leaves fall from the trees.
#In this way we can capture the difference between the same word used in a different context. For example we now have a
#different representation of the word “leaves” in the above two sentences

#IMPORTANT: parameteres in my case is random, we have to pay attention to select the right ones

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

def word_embeddings(final_corpus):
    X_embeddings_array = []
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(final_corpus)]
    print(tagged_data)
    model = Doc2Vec(size=200,
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
    model.build_vocab(tagged_data)
    max_epoch = 20
    for epoch in range(max_epoch):
        model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    X_embeddings_array = model.docvecs
    return X_embedding_array

#convert array to list

X_embedding_array_train = word_embeddings(final_train_corpus)
X_embedding_array_test = word_embeddings(final_test_corpus)

X_embeddings_train=[]
for i in range(len(X_embeddings_array_train)):
    X_embeddings_train.append(X_embeddings_array_train[i].tolist())
    
X_embeddings_test=[]
for i in range(len(X_embeddings_array_test)):
    X_embeddings_test.append(X_embeddings_array_test[i].tolist())
    

[TaggedDocument(words=['gas', 'by', 'my', 'house', 'hit', '339', 'im', 'go', 'to', 'chapel', 'hill', 'on', 'sat', '😂'], tags=['0']), TaggedDocument(words=['theo', 'walcott', 'be', 'still', 'shit', 'fan', 'watch', 'rafa', 'and', 'johnny', 'deal', 'with', 'him', 'on', 'saturday'], tags=['1']), TaggedDocument(words=['it', 'not', 'that', 'im', 'a', 'gsp', 'fan', 'i', 'just', 'hate', 'nick', 'diaz', 'cant', 'wait', 'for', 'february'], tags=['2']), TaggedDocument(words=['iranian', 'general', 'say', 'israel', 'iron', 'dome', 'cant', 'deal', 'with', 'their', 'missile', 'keep', 'talk', 'like', 'that', 'and', 'we', 'may', 'end', 'up', 'find', 'out'], tags=['3']), TaggedDocument(words=['tehran', 'mon', 'amour', 'obama', 'try', 'to', 'establish', 'tie', 'with', 'the', 'mullah', 'via', 'no', 'barack', 'obama', 'vote', 'mitt', 'romney'], tags=['4']), TaggedDocument(words=['i', 'sat', 'through', 'this', 'whole', 'movie', 'just', 'for', 'harry', 'and', 'ron', 'at', 'christmas', 'ohlawd', '911'], tags=

/home/dimitra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


NameError: name 'X_embedding_array' is not defined

In [9]:
#Add characteristics to embeddings
import pandas as pd
import csv
lexica =[pd.read_csv("lexica/affin/affin.txt", sep='\t', header=None),
              pd.read_csv("lexica/emotweet/valence_tweet.txt", sep='\t', header=None),
              pd.read_csv("lexica/generic/generic.txt", sep='\t', engine="python" ,quoting=csv.QUOTE_NONE,header=None),
              pd.read_csv("lexica/nrc/val.txt", sep='\t', engine="python",quoting=csv.QUOTE_NONE, header=None),
              pd.read_csv("lexica/nrctag/val.txt", sep='\t', header=None)]

#mean valence of each tweet
for tweet in range(len(train_tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(train_tokens[tweet])].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(train_tokens[tweet])][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:
             X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

#length of each tweet               
for tweet in range(len(train_tokens)):   #for each tweet
    X_embeddings[tweet].append(len(train_tokens[tweet]))


    
#min and max valence of each tweet
for tweet in range(len(train_tokens)):   #for each tweet
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(train_tokens[tweet])].empty: #search for tweets' tokens in lexicon
            max_valence = df[df[0].isin(train_tokens[tweet])][1].max() #get average sentiment for this tweet
            min_valence = df[df[0].isin(train_tokens[tweet])][1].min()
            X_embeddings[tweet].append(max_valence)
            X_embeddings[tweet].append(min_valence)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector

    
#mean valence for each half of a tweet
for tweet in range(len(train_tokens)):   #for each tweet
    entire_tweet = train_tokens[tweet]
    first_half= entire_tweet[:len(entire_tweet)//2]
    second_half= entire_tweet[len(entire_tweet)//2:]
    for df in lexica:         #for each lexicon
        if not df[df[0].isin(first_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(first_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        if not df[df[0].isin(second_half)].empty: #search for tweets' tokens in lexicon
            average_tweet_sentiment = df[df[0].isin(second_half)][1].mean() #get average sentiment for this tweet
            X_embeddings[tweet].append(average_tweet_sentiment)
        else:  # add two values to keep all the vectors the same size
            X_embeddings[tweet].append(0)   #if is zero should we add mean or it will affect the vector
        
    
            
for i in range(len(X_embeddings)):
    print(len(X_embeddings[i]))

#sentiment_sum = 0
#words_found = 0
#for j in range(len(tokens[i])):
#    if not df[df[0] == tokens[i][j]].empty: #search for tweets' tokens in lexicon
#        sentiment_sum += df[df[0] == tokens[i][j]].iloc[0][1] #get sentiment
#        words_found++
        

        


226
226
226
226
226
226
226
226


In [10]:
####################
#CONSTRUCT Y_LABELS#
####################

import pandas as pd
df = pd.read_csv("small.tsv", sep='\t', header=None)
y_labels = df[2].tolist() #sentiments


for n, value in enumerate(y_labels):
    if value == "positive":
        y_labels[n] = 2
    elif value =="negative":
        y_labels[n] = 0
    else:
        y_labels[n] =1

